In [1]:
%load_ext autoreload
%autoreload 2

import os
os.chdir('/projects/cft_vlm/finetune')

In [2]:
from transformers import AutoProcessor
import datasets
processor = AutoProcessor.from_pretrained("Qwen/Qwen2.5-VL-3B-Instruct", use_fast=True)

You have video processor config saved in `preprocessor.json` file which is deprecated. Video processor configs should be saved in their own `video_preprocessor.json` file. You can rename the file or load and save the processor back which renames it automatically. Loading from `preprocessor.json` will be removed in v5.0.


In [3]:
from qwenvl.data.vqa import VQADataset
from qwenvl.train.argument import DataArguments, ProcessingArguments

dataset = VQADataset(
  processor=processor,
  data_args=DataArguments(split='test'),
  proc_args=ProcessingArguments(add_generation_prompt=True),
  ds_key='flaviagiammarino/vqa-rad'
)

In [4]:
d = dataset[1:3]

In [7]:
dataset.make_conversation(dataset.ds.select(range(1)))

[{'role': 'user',
  'content': [{'type': 'image',
    'media': [<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1024x1291>]},
   {'type': 'text', 'text': ['is there evidence of an aortic aneurysm?']},
   {'type': 'text', 'text': 'Answer straightforwardly and concisely.'}]}]

In [ ]:
import json
import PIL.Image as PILImage
from io import BytesIO

def load_jsonl(example):
  return json.loads(example['jsonl'])

def filter_image(example):
  try:
    image = PILImage.open(BytesIO(example['jpg']['bytes']))
    image.verify()
    if image.height < 28 or image.width < 28:
      return False
  except Exception as e:
    print(f"Error processing image {example['image']}: {e}")
    return False
  
  return True

ds = (
  datasets.load_dataset('vector-institute/open-pmc', split='test')
  .map(load_jsonl, num_proc=32, remove_columns=['jsonl'])
  .cast_column('jpg', datasets.Image(decode=False))
  .filter(filter_image, num_proc=32)
  .cast_column('jpg', datasets.Image(decode=True))
)

Resolving data files:   0%|          | 0/350 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/44 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/44 [00:00<?, ?it/s]

Filter (num_proc=32):   0%|          | 0/218127 [00:00<?, ? examples/s]

Error processing image: cannot identify image file <_io.BytesIO object at 0x7f2616257e20>


In [16]:
ds[0]

{'jpg': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=210x363>,
 '__key__': 'test-0000-000000',
 '__url__': '/scratch/ssd004/scratch/xiaowenz/.cache/huggingface/hub/datasets--vector-institute--open-pmc/snapshots/a7add5e1b0d8659281931898b1377d27c0d1a2dd/test-0000.tar',
 'PMC_ID': 'PMC10060508',
 'image': 'test-0000-000000.jpg',
 'sub_caption': 'Postoperative MRI indicates no spinal cord compression and no spinal canal occupancy.',
 'full_caption': "Typical preoperative, postoperative, and follow-up images of severe thoracolumbar burst fractures with posterior midline spinal approach for spinal canal limited decompression and 13-mm titanium mesh implantation (case #2). (A,B) True lateral digital x-rays of the patient's thoracolumbar spine before the operation showing the L1 fracture. (C,D) True lateral digital x-rays of the thoracolumbar vertebrae after the operation showing good fixation and recovery of the height of the anterior edge of the L1 vertebra. (E,F) Preoperative magn

In [12]:

PILImage.open(BytesIO(ds[0]['jpg']))

TypeError: a bytes-like object is required, not 'dict'

In [11]:
class A:
  a: int = 1
  def __init__(self, q):
    self.a = q

a = A(1)
a.a

1

In [12]:
b = A(2)
b.a - a.a

1

In [3]:
from torch.utils.data import DataLoader
dataloader = DataLoader([1, 2, 3])

dataloader.dataset

[1, 2, 3]